<a href="https://colab.research.google.com/github/clustering-jun/KMU-Data_Science/blob/main/L12_Latent_Dirichlet_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Latent Dirichlet Allocation Practice**

## **데이터 불러오기**
- https://drive.google.com/file/d/1_tk2F1_ubaFEcF3GtM0sGDhObjnqr9Bs/view?usp=sharing

In [1]:
import pandas as pd

csv_path = "lda_korean_synthetic_corpus.csv"
df = pd.read_csv(csv_path)

print(df.shape)
df.head()

(2000, 2)


,text,topic_label
0,비가 왔지만 도쿄 관광지를 걸으며 여유를 느꼈다. 제주도 공항에서 산책로 수속을 빨...,여행/레저
1,분산학습 성능을 높이기 위해 NVIDIA와 분산학습를 함께 적용했다. TensorF...,기술/AI
2,금리 기대감에 코스닥 상당히 섹터 금리이(가) 확대됐다. 나스닥 채권를 분석하니 ...,금융/경제
3,주전 리그 컨디션이 좋아 농구 리그이(가) 상승세다. 원정 연장이(가) 이어졌지만 ...,스포츠
4,축구 감독 기록이 갱신되어 화제가 됐다. 테니스 수비 기록이 갱신되어 화제가 됐다....,스포츠


## **EDA: 샘플/통계/라벨 분포**

In [2]:
import numpy as np
print('컬럼:', df.columns.tolist())
print('결측치 개수:\n', df.isna().sum())
df['char_len'] = df['text'].str.len()
df['word_len'] = df['text'].str.split().apply(len)
print('\n길이 통계(문자):', df['char_len'].describe())
print('\n길이 통계(단어):', df['word_len'].describe())

if 'topic_label' in df.columns:
    print('\n라벨 분포:')
    print(df['topic_label'].value_counts())
df.head(3)

컬럼: ['text', 'topic_label']
결측치 개수:
 text           0
topic_label    0
dtype: int64

길이 통계(문자): count    2000.000000
mean       95.985500
std        36.166287
min        20.000000
25%        67.000000
50%        92.000000
75%       120.000000
max       259.000000
Name: char_len, dtype: float64

길이 통계(단어): count    2000.000000
mean       21.559000
std         7.924135
min         5.000000
25%        15.000000
50%        21.000000
75%        27.000000
max        55.000000
Name: word_len, dtype: float64

라벨 분포:
topic_label
여행/레저        224
금융/경제        210
건강/피트니스      209
아웃도어/자연      202
교육/학습        202
쇼핑/전자상거래     198
엔터테인먼트/문화    196
음식/요리        192
스포츠          192
기술/AI        175
Name: count, dtype: int64


,text,topic_label,char_len,word_len
0,비가 왔지만 도쿄 관광지를 걸으며 여유를 느꼈다. 제주도 공항에서 산책로 수속을 빨...,여행/레저,164,40
1,분산학습 성능을 높이기 위해 NVIDIA와 분산학습를 함께 적용했다. TensorF...,기술/AI,141,28
2,금리 기대감에 코스닥 상당히 섹터 금리이(가) 확대됐다. 나스닥 채권를 분석하니 ...,금융/경제,58,13


## **토큰화 & 벡터화**
- 간단한 한국어/영문 토큰 규칙(`2글자 이상`)
- 숫자는 `NUM`으로 치환
- `min_df`, `max_df`로 어휘 정제

In [3]:
import re
from sklearn.feature_extraction.text import CountVectorizer

token_re = re.compile(r"[가-힣]{2,}|[A-Za-z]{2,}")

def simple_tokenizer(text: str):
    text = re.sub(r"\d+", " NUM ", text)
    tokens = token_re.findall(text)
    tokens = [t.lower() for t in tokens if len(t) >= 2]
    return tokens

vectorizer = CountVectorizer(
    tokenizer=simple_tokenizer,
    token_pattern=None,
    max_df=0.95,
    min_df=5,
)
X = vectorizer.fit_transform(df['text'])
vocab = vectorizer.get_feature_names_out()
X.shape, len(vocab)

((2000, 867), 867)

## **LDA 학습**
- 토픽 수(`n_components`)를 여러 값으로 바꿔보세요.
- `learning_method`는 데이터 크기에 따라 `batch`/`online` 중 선택.

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 10  # 기본값 (데이터 생성 시 주제 10개)
lda = LatentDirichletAllocation(
    n_components=n_topics,
    learning_method='batch',
    max_iter=30,
    random_state=42,
    evaluate_every=5,
)
lda.fit(X)
print('학습 완료')

학습 완료


### **토픽별 상위 단어 보기**

In [5]:
import numpy as np

def print_top_words(model, feature_names, n_top_words=12):
    for topic_idx, topic in enumerate(model.components_):
        top_idx = topic.argsort()[::-1][:n_top_words]
        words = [feature_names[i] for i in top_idx]
        print(f"[토픽 {topic_idx}] ", ", ".join(words))

print_top_words(lda, vocab, n_top_words=12)

[토픽 0]  실적, 섹터, 확대됐다, 분석하니, 기대감에, 엇갈렸다, 전망이, 투자자는, 분산으로, 변동성에, 대비했다, 관련
[토픽 1]  속에서, 텐트를, 코스를, 감상했다, 걸으며, 설치했다, 초저녁, 준비, 즐겼다, 조망이, 훌륭했다, 맑은
[토픽 2]  루틴으로, 좋아졌다, 꾸준한, 퍼포먼스가, 가벼운, 세션을, 시작해, 마무리했다, 관리, 덕분에, 기록이, 향상됐다
[토픽 3]  구매하고, 읽고, 적용했다, 이벤트로, 사용자, 결정했다, 높았다, 만족도가, 빨라서, 배송이, 인기가, 상태를
[토픽 4]  비가, 걸으며, 왔지만, 느꼈다, 여유를, 떠나, 찍기, 근처, 끝내고, 공항에서, 즐겼다, 예약을
[토픽 5]  살아났다, 없이, 지켜, 포인트를, 완성됐다, 실패, 활용한, 레시피로, 간단하지만, 만들었다, 함께, 덕분에
[토픽 6]  남겼다, 굿즈, 사진도, 부스에서, 무대에서, 구입하고, 인상적이었다, 연출이, 이뤘다, 감상을, 조화를, 신작
[토픽 7]  흐름을, 됐다, 바꿨다, 감독의, 좋아, 원정, 주전, 극복했다, 팀워크로, 상승세다, 변화가, 이어졌지만
[토픽 8]  성능을, 높이기, 적용했다, 위해, 함께, 일정을, 클러스터의, 실험에서, 자원이, 부족해, 대규모, 조정했다
[토픽 9]  주제를, 다음, 탐구했다, 준비했다, 대비해, 기말, 참고해, 이후, 품질을, 자료를, 피드백을, 끌어올렸다


### **문서-토픽 분포 보기**

In [6]:
doc_topic = lda.transform(X)
import pandas as pd
sample_idx = np.random.choice(len(df), 10, replace=False)
sample = pd.DataFrame({
    'text': df.iloc[sample_idx]['text'].str.slice(0, 120) + '...',
    'true_label': df.iloc[sample_idx]['topic_label'] if 'topic_label' in df.columns else None,
    'pred_dominant_topic': doc_topic[sample_idx].argmax(axis=1),
    'topic_dist': [np.array2string(r, precision=3, separator=', ') for r in doc_topic[sample_idx]]
})
sample

,text,true_label,pred_dominant_topic,topic_dist
152,다음 주 시험를 대비해 튜터 시험을(를) 준비했다. 기말 발표 이후 세미나실 발표를...,교육/학습,9,"[0.005, 0.005, 0.108, 0.005, 0.005, 0.005, 0.0..."
1425,헬스장에서 체지방 위주로 운동하고 체지방을(를) 보충했다. 꾸준한 컨디션 루틴으로 ...,건강/피트니스,2,"[0.005, 0.059, 0.899, 0.005, 0.005, 0.005, 0.0..."
618,신작 예매 예매가 빠르게 매진돼 갤러리가 화제다. 갤러리에서 관람을(를) 관람하고 ...,엔터테인먼트/문화,6,"[0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.9..."
1923,초저녁 캠핑 속 아주 에서 태백산 텐트를 설치했다. 맑은 꽤 날씨에 지리산 일몰 ...,아웃도어/자연,1,"[0.005, 0.955, 0.005, 0.005, 0.005, 0.005, 0.0..."
390,초저녁 불멍 속에서 태백산 상당히 텐트를 설치했다. 야영지에서 바람 준비 후 무등...,아웃도어/자연,1,"[0.004, 0.965, 0.004, 0.004, 0.004, 0.004, 0.0..."
110,된장을(를) 활용한 스테이크 레시피로 스테이크을(를) 꽤 만들었다. 마늘을(를) ...,음식/요리,5,"[0.007, 0.007, 0.007, 0.007, 0.007, 0.936, 0.0..."
1018,아트하우스에서 관람을(를) 관람하고 감상을 남겼다. 아트하우스 무대에서 굿즈과(와)...,엔터테인먼트/문화,6,"[0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.9..."
1688,강릉 온천 근처 온천가 사진 찍기 좋았다. 부산 공항에서 숙소 수속을 빨리 끝내고 ...,여행/레저,4,"[0.003, 0.003, 0.003, 0.003, 0.976, 0.003, 0.0..."
1964,대규모 파이썬 실험에서 NVIDIA 클러스터의 자원이 부족해 파이썬 일정을 조정했다...,기술/AI,8,"[0.003, 0.003, 0.003, 0.003, 0.003, 0.003, 0.0..."
1009,덕유산 조망 코 아주 스를 걸으며 조망을(를) 감상했다. 태백산 버너 코스를 걸으며...,아웃도어/자연,1,"[0.005, 0.955, 0.005, 0.005, 0.005, 0.005, 0.0..."


## **토픽 일관성(UMass) 지표 간단 계산**
- UMass 코히런스는 동일 말뭉치에서의 단어 공기(동시 출현)를 사용함.

In [7]:
from collections import Counter

def umass_coherence(top_words, X, vocab):
    # 단순 동시 출현 기반 (문서 레벨)
    word_to_idx = {w:i for i, w in enumerate(vocab)}
    # 단어 -> 등장 문서 집합
    docs_containing = {}
    for w in top_words:
        if w in word_to_idx:
            j = word_to_idx[w]
            docs_containing[w] = set(X[:, j].nonzero()[0])
        else:
            docs_containing[w] = set()
    # UMass 평균
    import math
    score = 0.0
    cnt = 0
    for i in range(1, len(top_words)):
        for j in range(0, i):
            wi, wj = top_words[i], top_words[j]
            Di = len(docs_containing[wi]) + 1e-12
            Dij = len(docs_containing[wi] & docs_containing[wj]) + 1
            score += math.log(Dij/Di)
            cnt += 1
    return score / max(cnt,1)

def topic_coherence_table(lda, vocab, X, n_top=10):
    rows = []
    for k, comp in enumerate(lda.components_):
        top_idx = comp.argsort()[::-1][:n_top]
        words = [vocab[i] for i in top_idx]
        coh = umass_coherence(words, X, vocab)
        rows.append({'topic': k, 'coherence_umass': coh, 'top_words': ', '.join(words)})
    return pd.DataFrame(rows).sort_values('coherence_umass', ascending=False)

coh_df = topic_coherence_table(lda, vocab, X, n_top=10)
coh_df

,topic,coherence_umass,top_words
5,5,-0.435272,"살아났다, 없이, 지켜, 포인트를, 완성됐다, 실패, 활용한, 레시피로, 간단하지만..."
8,8,-0.467519,"성능을, 높이기, 적용했다, 위해, 함께, 일정을, 클러스터의, 실험에서, 자원이,..."
6,6,-0.498419,"남겼다, 굿즈, 사진도, 부스에서, 무대에서, 구입하고, 인상적이었다, 연출이, 이..."
2,2,-0.577485,"루틴으로, 좋아졌다, 꾸준한, 퍼포먼스가, 가벼운, 세션을, 시작해, 마무리했다, ..."
0,0,-0.601198,"실적, 섹터, 확대됐다, 분석하니, 기대감에, 엇갈렸다, 전망이, 투자자는, 분산으..."
4,4,-0.602816,"비가, 걸으며, 왔지만, 느꼈다, 여유를, 떠나, 찍기, 근처, 끝내고, 공항에서"
7,7,-0.655298,"흐름을, 됐다, 바꿨다, 감독의, 좋아, 원정, 주전, 극복했다, 팀워크로, 상승세다"
3,3,-0.682503,"구매하고, 읽고, 적용했다, 이벤트로, 사용자, 결정했다, 높았다, 만족도가, 빨라..."
9,9,-0.705708,"주제를, 다음, 탐구했다, 준비했다, 대비해, 기말, 참고해, 이후, 품질을, 자료를"
1,1,-0.869370,"속에서, 텐트를, 코스를, 감상했다, 걸으며, 설치했다, 초저녁, 준비, 즐겼다, 조망이"


## **하이퍼파라미터 그리드 탐색**
- 간단한 격자 탐색 예시

In [10]:
from sklearn.model_selection import ParameterGrid
params = {
    'n_components': [8, 10, 12],
    'learning_decay': [0.7, 0.9],
}
grid = list(ParameterGrid(params))
results = []
for p in grid:
    lda_g = LatentDirichletAllocation(
        **p, learning_method='batch', max_iter=20, random_state=0
    ).fit(X)
    # 평가 지표 예: 퍼플렉서티(작을수록 좋음)와 간단 코히런스(클수록 좋음)의 조합을 참고
    perp = lda_g.perplexity(X)
    coh = topic_coherence_table(lda_g, vocab, X, n_top=10)['coherence_umass'].mean()
    results.append({'params': p, 'perplexity': perp, 'coherence_umass_mean': coh})
pd.DataFrame(results).sort_values(['coherence_umass_mean', 'perplexity'], ascending=[False, True])

,params,perplexity,coherence_umass_mean
2,"{'learning_decay': 0.7, 'n_components': 12}",100.125881,-0.759245
5,"{'learning_decay': 0.9, 'n_components': 12}",100.125881,-0.759245
1,"{'learning_decay': 0.7, 'n_components': 10}",110.227934,-0.797857
4,"{'learning_decay': 0.9, 'n_components': 10}",110.227934,-0.797857
0,"{'learning_decay': 0.7, 'n_components': 8}",128.738214,-1.042185
3,"{'learning_decay': 0.9, 'n_components': 8}",128.738214,-1.042185
